# Predict Cytarabine response using gene expression groups (low/high) by perfoming logistic regression and SST

## Load libraries

In [1]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [2]:
import joblib
import math
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import sklearn
from sklearn.preprocessing import StandardScaler
from sklearn.utils import shuffle
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.model_selection import StratifiedKFold, KFold, GridSearchCV, cross_val_score, train_test_split

import tensorflow as tf
from tensorflow import keras

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation
from tensorflow.keras import layers, models, utils, losses

from tensorflow.keras.regularizers import l2
#from keras.optimizers import gradient_descent_v2
from tensorflow.keras.optimizers import SGD, RMSprop, Adam
from tensorflow.keras.optimizers.experimental import AdamW

from tensorflow.keras.callbacks import LearningRateScheduler, History

import keras_tuner

2023-11-03 12:36:32.596695: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-03 12:36:32.633027: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-03 12:36:32.633074: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-03 12:36:32.633106: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-03 12:36:32.639455: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-03 12:36:32.639821: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

Using TensorFlow backend


## Define data directories

In [3]:
dir_path = '/Keras-tuner'
tsv_file = '/Gene-Classes/CCLE_Lab_NatCommValidation_TMM-Counts-2Groups-1-3_IC50-2Grp_NEW-FOR-LAB-PRIMARY-VALIDATION_NO-TT.tsv'
gridcv_file = '/selectedModels/gridsearchcv_logReg.pkl'
logreg_gene_file = '/selectedFeatures/gridsearchcv_L1_selectedFeatures.txt'
save_model_dir = '/Gene-Classes/Keras-tuner/kerasTuner_SSTmodel'

## Default parameters

In [4]:
seed = 1
np.random.seed(seed)

njobs = 14

epochs = 5

## Define specific and rest cancer samples

In [5]:
# specificCancerTest_List: cell lines from the cancer of interest
specificCancerTest_List = ['F-36P_3',
                           'HEL',
                           'HL60',
                           'KASUMI-1_1',
                           'KASUMI-1_2',
                           'MV411',
                           'MONOMAC6',
                           'NB4',
                           'NOMO-1_2',
                           'NOMO-1_3',
                           'OCI-AML2_1',
                           'OCI-AML2_2',
                           'OCI-AML2_3'
                          ]

restCancerTest_List = ['BV173',
                       'COLO668',
                       'DU4475',
                       'GRANTA519',
                       'HCC15',
                       'HPAC',
                       'HT144',
                       'KU1919',
                       'MKN1',
                       'NB1',
                       'NCIH1048',
                       'OV7',
                       'RPMI8226'
                      ]

specificCancerAll_List = ['CMK',
                          'EOL1',
                          'F-36P_1',
                          'F-36P_2',
                          'F-36P_3',
                          'GDM1',
                          'HEL',
                          'HL60',
                          'HL-60_1',
                          'HL-60_2',
                          'HL-60_3',
                          'KASUMI1',
                          'KASUMI-1_1',
                          'KASUMI-1_2',
                          'KASUMI-1_3',
                          'KG1',
                          'ME1',
                          'MOLM13',
                          'MOLM-13_1',
                          'MOLM-13_2',
                          'MOLM-13_3',
                          'MOLM16',
                          'MONOMAC6',
                          'MONO-MAC-6_1',
                          'MONO-MAC-6_2',
                          'MONO-MAC-6_3',
                          'MV411',
                          'MV-4-11_1',
                          'MV-4-11_2',
                          'MV-4-11_3',
                          'NB4',
                          'NB-4_1',
                          'NB-4_2',
                          'NB-4_3',
                          'NOMO1',
                          'NOMO-1_1',
                          'NOMO-1_2',
                          'NOMO-1_3',
                          'OCIAML2',
                          'OCI-AML2_1',
                          'OCI-AML2_2',
                          'OCI-AML2_3',
                          'OCIAML3',
                          'OCI-AML3_1',
                          'OCI-AML3_2',
                          'OCI-AML3_3',
                          'OCIAML5',
                          'OCIM1',
                          'PL21',
                          'SKM1',
                         'THP-1_1',
                          'THP-1_2',
                          'THP-1_3'
                         ]
# validationAMLNatComm_List: primary AML from Nature Comm (PMID: 29298978)
validationAMLNatComm_List = ['AMLS1',
                             'AMLS1.1',
                             'AMLS2',
                             'AMLS2.1',
                             'AMLS3',
                             'AMLS3.1',
                             'AMLS4',
                             'AMLS4.1',
                             'AMLS5',
                             'AMLS5.1',
                             'AMLS6',
                             'AMLS6.1',
                             'AMLS7',
                             'AMLS7.1',
                             'AMLS8',
                             'AMLS8.1',
                             'AMLS9',
                             'AMLS9.1',
                             'AMLS10',
                             'AMLS10.1',
                             'AMLS11',
                             'AMLS11.1',
                             'AMLS12',
                             'AMLS12.1']

# Here total number of AML samples=53, AML samples to be used for testing=13, total AML samples for training=40

## Functions

In [6]:
def load_dataset(dir_path,
                 tsv_file):
    
    dataset = pd.read_csv(tsv_file,
                          sep="\t")
    
    return dataset

## Function to split datasets into training, test, and validation sets
def generate_train_test_val(dataset,
                            specific_test,
                            rest_test,
                            val):
    
    dataset_train = dataset[~dataset['Cell'].isin(specific_test + rest_test + val)].reset_index(drop=True)
    dataset_test = dataset[dataset['Cell'].isin(specific_test + rest_test)].reset_index(drop=True)
    dataset_val = dataset[dataset['Cell'].isin(val)].reset_index(drop=True)
    
    return dataset_train, dataset_test, dataset_val

def generate_x_y(dataset_train):
    
    X = dataset_train.iloc[:, 1:-1]
    y = dataset_train.iloc[:, -1]-1
    
    X.index = dataset_train["Cell"]
    # new
    y.index = dataset_train["Cell"]
    # new-end
    scaler = StandardScaler()
    
    X_std = scaler.fit_transform(X)
    X_std = pd.DataFrame(X_std)
    
    X_std.columns = X.columns.to_list()
    X_std.index = X.index.to_list()
    
    return X_std, y

## Function to select specific genes
def select_from_dataset(dataset_train,
                   dataset_test,
                   dataset_val,
                   imp_genes):
    
    dataset_train_selected = dataset_train[imp_genes]
    dataset_test_selected = dataset_test[imp_genes]
    dataset_val_selected = dataset_val[imp_genes]
    
    return dataset_train_selected, dataset_test_selected, dataset_val_selected

def generate_x_y_from_selected(dataset_train_selected,
                               dataset_test_selected,
                               dataset_val_selected,
                               specific_test,
                               rest_test):
    
    scaler = StandardScaler()
    
    # Train
    X_train_select = dataset_train_selected.iloc[:, 1:-1]
    y_train_select = dataset_train_selected.iloc[:, -1]-1
    X_train_select.index = dataset_train_selected["Cell"]
    # new
    y_train_select.index = dataset_train_selected["Cell"]
    #new-end
    
    X_train_select_std = scaler.fit_transform(X_train_select)
    X_train_select_std = pd.DataFrame(X_train_select_std)
    X_train_select_std.columns = X_train_select.columns.to_list()
    X_train_select_std.index = X_train_select.index.to_list()
    
    X_train_select_std_split,X_test_select_std_split,y_train_select_split,y_test_select_split = train_test_split(X_train_select_std,
                              y_train_select,
                              test_size=0.3,
                              random_state=1,
                              stratify=y)
    
    # Test
    X_test_select = dataset_test_selected.iloc[:, 1:-1]
    y_test_select = dataset_test_selected.iloc[:, -1]-1
    X_test_select.index = dataset_test_selected["Cell"]
    # new
    y_test_select.index = dataset_test_selected["Cell"]
    # new-end
    
    X_test_select_std = scaler.fit_transform(X_test_select)
    X_test_select_std = pd.DataFrame(X_test_select_std)
    X_test_select_std.columns = X_test_select.columns.to_list()
    X_test_select_std.index = X_test_select.index.to_list()
    
    ## Test - specific and rest cancers
    splitSet = {'SPECIFIC': specific_test, 'REST': rest_test}
    X_test_select_SPECIFIC = X_test_select.iloc[X_test_select.index.isin(splitSet.get('SPECIFIC'))] 
    y_test_select_SPECIFIC = dataset_test_selected.values[dataset_test_selected['Cell'].isin(splitSet.get('SPECIFIC')),dataset_test_selected.shape[1]-1]-1
    #old y_test_select_SPECIFIC = pd.to_numeric(y_test_select_SPECIFIC)
    # new
    y_test_select_SPECIFIC = pd.DataFrame(y_test_select_SPECIFIC)
    y_test_select_SPECIFIC.index = X_test_select_SPECIFIC.index
    y_test_select_SPECIFIC.columns = ["Group"]
    # new-wnd
    
    X_test_select_SPECIFIC_std = scaler.fit_transform(X_test_select_SPECIFIC)
    X_test_select_SPECIFIC_std = pd.DataFrame(X_test_select_SPECIFIC_std)
    X_test_select_SPECIFIC_std.columns = X_test_select_SPECIFIC.columns.to_list()
    X_test_select_SPECIFIC_std.index = X_test_select_SPECIFIC.index.to_list()
    
    X_test_select_REST = X_test_select.iloc[X_test_select.index.isin(splitSet.get('REST'))]
    y_test_select_REST = dataset_test_selected.values[dataset_test_selected['Cell'].isin(splitSet.get('REST')),dataset_test_selected.shape[1]-1]-1
    #old y_test_select_REST = pd.to_numeric(y_test_select_REST)
    # new
    y_test_select_REST = pd.DataFrame(y_test_select_REST)
    y_test_select_REST.index = X_test_select_REST.index
    y_test_select_REST.columns = ["Group"]
    # new-wnd
    
    X_test_select_REST_std = scaler.fit_transform(X_test_select_REST)
    X_test_select_REST_std = pd.DataFrame(X_test_select_REST_std)
    X_test_select_REST_std.columns = X_test_select_REST.columns.to_list()
    X_test_select_REST_std.index = X_test_select_REST.index.to_list()
    
    # Val
    X_val_select = dataset_val_selected.iloc[:, 1:-1]
    y_val_select = dataset_val_selected.iloc[:, -1]-1
    X_val_select.index = dataset_val_selected["Cell"]
    
    y_val_select.index = dataset_val_selected["Cell"]
    
    X_val_select_std = scaler.fit_transform(X_val_select)
    X_val_select_std = pd.DataFrame(X_val_select_std)
    X_val_select_std.columns = X_val_select.columns.to_list()
    X_val_select_std.index = X_val_select.index.to_list()
    
    return X_train_select_std_split, y_train_select_split, X_test_select_std_split, y_test_select_split, X_test_select_std, y_test_select, X_test_select_SPECIFIC_std, y_test_select_SPECIFIC, X_test_select_REST_std, y_test_select_REST, X_val_select_std, y_val_select

## Generate datasets

In [7]:
dataset = load_dataset(dir_path=dir_path,
                       tsv_file=tsv_file)

dataset_train, dataset_test, dataset_val = generate_train_test_val(dataset=dataset,
                                                                   specific_test=specificCancerTest_List,
                                                                   rest_test=restCancerTest_List,
                                                                   val=validationAMLNatComm_List)

X_std, y = generate_x_y(dataset_train=dataset_train)

/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future ver

## Logistic regression

### Define kfolds

In [8]:
skf = StratifiedKFold(n_splits=10, shuffle=True, random_state=1)

### Define grid parameters

In [ ]:
grid = {"C":np.logspace(-3,3,7),
      "penalty":["l1"],
      "solver": ["liblinear", "saga"],
      "max_iter":[100,200,500,1000]}

logreg=LogisticRegression()
logreg_cv=GridSearchCV(logreg,grid,cv=skf, n_jobs=njobs)
logreg_cv.fit(X_std, y)

### Print best parameters

In [ ]:
print("tuned hpyerparameters :(best parameters) ",logreg_cv.best_params_)
print("accuracy :",logreg_cv.best_score_)

### Write gridcv parameters to the directory

In [ ]:
joblib.dump(logreg_cv, dir_path + 'expression-binary-1-3-lasso-Grid.pkl')

### Load previous gridcv parameters from the directory

In [9]:
logreg_cv = joblib.load(gridcv_file)

### Logistic regression using griddcv-optimized parameters

In [10]:
model_lr = LogisticRegression(**logreg_cv.best_params_, n_jobs=njobs)
model_lr.fit(X_std, y)

/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future ver

LogisticRegression(C=10.0, max_iter=200, n_jobs=14, penalty='l1',
                   solver='liblinear')

#### Remove the coefficients (genes) with 0 values

In [11]:
coef = model_lr.coef_[0]

imp_genes_lr = X_std.columns.values[coef!=0].tolist()
imp_genes_lr_df = pd.DataFrame(imp_genes_lr, columns = ['Genes'])

#### Write logregcv-selected genes to the directory

In [ ]:
imp_genes_lr_df.to_csv(dir_path + "expression-binary-1-3-groups_Cyt-2Grp-Regularization-ANN_CCLE.csv",
                       sep="\t")

with open(dir_path + "expression-binary-1-3-groups_Cyt-2Grp-Regularization-ANN_CCLE.txt", "w",) as txt_file:
    txt_file.write(str(imp_mut_lr))

#### Load previous logregcv-selected genes

In [12]:
with open(logreg_gene_file,'r') as f:
    imp_genes_lr = eval(f.read())

#### Append 'Cell' entry to the imp_genes list

In [13]:
imp_genes_lr.insert(0, 'Cell')
imp_genes_lr.append('Group')

### Validation - logreg

#### Generate X, y datasets for validation

In [14]:
dataset_train_selected,dataset_test_selected,dataset_val_selected = select_from_dataset(dataset_train=dataset_train,
                                           dataset_test=dataset_test,
                                           dataset_val=dataset_val,
                                           imp_genes=imp_genes_lr)

X_train_select_std_split,y_train_select_split,X_test_select_std_split,y_test_select_split,X_test_select_std,y_test_select,X_test_select_SPECIFIC_std,y_test_select_SPECIFIC,X_test_select_REST_std,y_test_select_REST,X_val_select_std,y_val_select = generate_x_y_from_selected(dataset_train_selected=dataset_train_selected,
                                         dataset_test_selected=dataset_test_selected,
                                         dataset_val_selected=dataset_val_selected,
                                         specific_test=specificCancerTest_List,
                                         rest_test=restCancerTest_List)

/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future ver

#### Split validation

In [35]:
model_lr_val = LogisticRegression(**logreg_cv.best_params_, n_jobs=njobs)
model_lr_val.fit(X_train_select_std_split, y_train_select_split)

y_pred_lr_split = model_lr_val.predict(X_test_select_std_split)

print("Prediction: ", sklearn.metrics.accuracy_score(y_test_select_split, y_pred_lr_split))
#print("Precision: ", sklearn.metrics.precision_score(y_test_select_split, y_pred_lr_split))
#print("Recall: ", sklearn.metrics.recall_score(y_test_select_split, y_pred_lr_split))

cnf_matrix = sklearn.metrics.confusion_matrix(y_test_select_split, y_pred_lr_split)

Prediction:  0.7684729064039408


/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future ver

#### Separate validation

In [36]:
y_pred_lr = model_lr_val.predict(X_test_select_std)

print("Prediction: ", sklearn.metrics.accuracy_score(y_test_select, y_pred_lr))
#print("Precision: ", sklearn.metrics.precision_score(y_test_select, y_pred_lr))
#print("Recall: ", sklearn.metrics.recall_score(y_test_select, y_pred_lr))

Prediction:  0.5769230769230769


/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future ver

#### Separate validation - Specific cancer

In [49]:
y_pred_lr_SPECIFIC = model_lr_val.predict(X_test_select_SPECIFIC_std)

print("Prediction: ", sklearn.metrics.accuracy_score(y_test_select_SPECIFIC['Group'].astype(np.int64), y_pred_lr_SPECIFIC))
#print("Precision: ", sklearn.metrics.precision_score(y_test_select_SPECIFIC, y_pred_lr_SPECIFIC))
#print("Recall: ", sklearn.metrics.recall_score(y_test_select_SPECIFIC, y_pred_lr_SPECIFIC))

Prediction:  0.46153846153846156


/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future ver

#### Separate validation - Rest cancer

In [50]:
y_pred_lr_REST = model_lr_val.predict(X_test_select_REST_std)

print("Prediction: ", sklearn.metrics.accuracy_score(y_test_select_REST.astype(np.int64), y_pred_lr_REST))
#print("Precision: ", sklearn.metrics.precision_score(y_test_select_REST, y_pred_lr_REST))
#print("Recall: ", sklearn.metrics.recall_score(y_test_select_REST, y_pred_lr_REST))

Prediction:  0.7692307692307693


/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future ver

#### Separate validation - Primary AML

In [21]:
y_pred_val_lr = model_lr_val.predict(X_val_select_std)

print("Prediction: ", sklearn.metrics.accuracy_score(y_val_select, y_pred_val_lr))
#print("Precision: ", sklearn.metrics.precision_score(y_val_select, y_pred_val_lr))
#print("Recall: ", sklearn.metrics.recall_score(y_val_select, y_pred_val_lr))

Prediction:  0.5416666666666666


/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:767: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if not hasattr(array, "sparse") and array.dtypes.apply(is_sparse).any():
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/home/t14s/anaconda3/envs/AI/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future ver

## Neural network with HyperTuner

#### Define hp search parameters and model

In [22]:
# Define search parameter for the Keras_Tuner
hp = keras_tuner.HyperParameters()

def build_model_hyper(hp):
    
    max_neurons = X_train_select_std_split.shape[1]
    
    model = tf.keras.Sequential()
  
    # Tune the number of units in the first Dense layer

    hp_units1 = hp.Int('units1', min_value=100, max_value=max_neurons, step=100)
    hp_units2 = hp.Int('units2', min_value=100, max_value=max_neurons, step=100)
    hp_units3 = hp.Int('units3', min_value=100, max_value=max_neurons, step=100)
    
    model.add(Dense(
        units=hp_units1,
        kernel_initializer=hp.Choice("initializer", ['uniform', 'normal', 'he_uniform', 'he_normal']),
        activation='relu'))
    
    model.add(Dropout(hp.Choice("dropout1", [0.1,0.2,0.3, 0.4, 0.5])))
    
    model.add(tf.keras.layers.Dense(units=hp_units2, activation='relu'))
    
    model.add(Dropout(hp.Choice("dropout2", [0.1,0.2,0.3, 0.4, 0.5])))
    
    model.add(tf.keras.layers.Dense(units=hp_units3, activation='relu'))
    
    model.add(Dense(1, activation=hp.Choice("activation", ['sigmoid', 'softmax'])))
    # Tune the learning rate for the optimizer
    # Choose an optimal value from 0.01, 0.001, or 0.0001
    
    hp_learning_rate = hp.Choice('learning_rate', values=[1e-1, 1e-2, 1e-3, 1e-4, 1e-5])
    
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=hp_learning_rate),
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    return model

#### Keras Tuner - RandomSearch

In [23]:
tuner = keras_tuner.RandomSearch(
    build_model_hyper,
    #objective='val_loss',
    objective='val_accuracy',
    max_trials=10,
    overwrite = True,
    seed=seed,
    directory='kerasTuner_model_RandomSearch',
    project_name='nn_kerastuner')

# Or use keras_tuner.BayesianOptimization

tuner.search(X_train_select_std_split,
                          y_train_select_split-1,
                          epochs=epochs,
                          validation_data=(X_test_select_std_split,
                                           y_test_select_split-1)) # Test using split test data or AML-specific data???

best_model = tuner.get_best_models()[0]
best_hp = tuner.get_best_hyperparameters()[0]
print(best_hp.values)

model_RandomSearch = tuner.hypermodel.build(best_hp)
history_RandomSearch = model_RandomSearch.fit(X_train_select_std_split,
                                  y_train_select_split-1,
                                  epochs=150,
                                  validation_data=(X_test_select_std_split,
                                                   y_test_select_split-1),
                                  use_multiprocessing=True,
                                  workers=njobs)

val_acc_per_epoch = history_RandomSearch.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

# Re-instantiate the hypermodel and train it with the optimal number of epochs from above.
hypermodel = tuner.hypermodel.build(best_hp)

# Retrain the model
history_RandomSearch_hyper = hypermodel.fit(X_train_select_std_split,
                                                         y_train_select_split-1,
                                                         epochs=best_epoch, 
                                                         validation_data=(X_test_select_std_split,
                                                                          y_test_select_split-1))

Trial 10 Complete [00h 00m 02s]
val_accuracy: 0.5320196747779846

Best val_accuracy So Far: 0.5320196747779846
Total elapsed time: 00h 00m 18s
{'units1': 200, 'units2': 1200, 'units3': 300, 'initializer': 'uniform', 'dropout1': 0.4, 'dropout2': 0.4, 'activation': 'sigmoid', 'learning_rate': 0.01}
Epoch 1/150
15/15 [==============================] - 1s 17ms/step - loss: -25256.3926 - accuracy: 0.5011 - val_loss: -135910.1562 - val_accuracy: 0.5320
Epoch 2/150
15/15 [==============================] - 0s 9ms/step - loss: -1323874.2500 - accuracy: 0.5349 - val_loss: -3437882.0000 - val_accuracy: 0.5320
Epoch 3/150
15/15 [==============================] - 0s 9ms/step - loss: -14601308.0000 - accuracy: 0.5349 - val_loss: -27377260.0000 - val_accuracy: 0.5320
Epoch 4/150
15/15 [==============================] - 0s 9ms/step - loss: -86443072.0000 - accuracy: 0.5349 - val_loss: -129260064.0000 - val_accuracy: 0.5320
Epoch 5/150
15/15 [==============================] - 0s 9ms/step - loss: -33377

15/15 [==============================] - 0s 9ms/step - loss: -38967579770880.0000 - accuracy: 0.5349 - val_loss: -28723313115136.0000 - val_accuracy: 0.5320
Epoch 49/150
15/15 [==============================] - 0s 9ms/step - loss: -42550777872384.0000 - accuracy: 0.5349 - val_loss: -31272032272384.0000 - val_accuracy: 0.5320
Epoch 50/150
15/15 [==============================] - 0s 9ms/step - loss: -46533806391296.0000 - accuracy: 0.5349 - val_loss: -33994089431040.0000 - val_accuracy: 0.5320
Epoch 51/150
15/15 [==============================] - 0s 9ms/step - loss: -50158435827712.0000 - accuracy: 0.5349 - val_loss: -36745273933824.0000 - val_accuracy: 0.5320
Epoch 52/150
15/15 [==============================] - 0s 9ms/step - loss: -53871892234240.0000 - accuracy: 0.5349 - val_loss: -39791722758144.0000 - val_accuracy: 0.5320
Epoch 53/150
15/15 [==============================] - 0s 9ms/step - loss: -58489934184448.0000 - accuracy: 0.5349 - val_loss: -42991938961408.0000 - val_accuracy: 

Epoch 96/150
15/15 [==============================] - 0s 9ms/step - loss: -599086063419392.0000 - accuracy: 0.5349 - val_loss: -427846992396288.0000 - val_accuracy: 0.5320
Epoch 97/150
15/15 [==============================] - 0s 9ms/step - loss: -621046835183616.0000 - accuracy: 0.5349 - val_loss: -444840030502912.0000 - val_accuracy: 0.5320
Epoch 98/150
15/15 [==============================] - 0s 9ms/step - loss: -640539611365376.0000 - accuracy: 0.5349 - val_loss: -462162774458368.0000 - val_accuracy: 0.5320
Epoch 99/150
15/15 [==============================] - 0s 9ms/step - loss: -670977910374400.0000 - accuracy: 0.5349 - val_loss: -479508469645312.0000 - val_accuracy: 0.5320
Epoch 100/150
15/15 [==============================] - 0s 10ms/step - loss: -689510090276864.0000 - accuracy: 0.5349 - val_loss: -497507301851136.0000 - val_accuracy: 0.5320
Epoch 101/150
15/15 [==============================] - 0s 9ms/step - loss: -719813030707200.0000 - accuracy: 0.5349 - val_loss: -516187926

15/15 [==============================] - 0s 9ms/step - loss: -2556920195973120.0000 - accuracy: 0.5349 - val_loss: -1804748613222400.0000 - val_accuracy: 0.5320
Epoch 144/150
15/15 [==============================] - 0s 10ms/step - loss: -2605035103977472.0000 - accuracy: 0.5349 - val_loss: -1849965894696960.0000 - val_accuracy: 0.5320
Epoch 145/150
15/15 [==============================] - 0s 9ms/step - loss: -2698292802617344.0000 - accuracy: 0.5349 - val_loss: -1895293637361664.0000 - val_accuracy: 0.5320
Epoch 146/150
15/15 [==============================] - 0s 9ms/step - loss: -2734424751865856.0000 - accuracy: 0.5349 - val_loss: -1939583474335744.0000 - val_accuracy: 0.5320
Epoch 147/150
15/15 [==============================] - 0s 9ms/step - loss: -2791283944849408.0000 - accuracy: 0.5349 - val_loss: -1987558393249792.0000 - val_accuracy: 0.5320
Epoch 148/150
15/15 [==============================] - 0s 9ms/step - loss: -2890542249672704.0000 - accuracy: 0.5349 - val_loss: -20355974

#### Split Validation

In [53]:
score_NN_RandomSearch = hypermodel.evaluate(X_test_select_std_split,
                                                         y_test_select_split,
                                                         verbose=0)

print("Split validation - Loss: ", score_NN_RandomSearch[0])
print("Split validation - Accuracy: ", score_NN_RandomSearch[1])

Split validation - Loss:  42577.58984375
Split validation - Accuracy:  0.467980295419693


#### Sparate Validation

In [54]:
score_NN_RandomSearch = hypermodel.evaluate(X_test_select_std,
                                                         y_test_select,
                                                         verbose=0)

print("Separate validation - Loss: ", score_NN_RandomSearch[0])
print("Separate validation - Accuracy: ", score_NN_RandomSearch[1])

Separate validation - Loss:  94206.0859375
Separate validation - Accuracy:  0.6153846383094788


#### Sparate Validation - Specific cancer

In [56]:
score_NN_RandomSearch = hypermodel.evaluate(X_test_select_SPECIFIC_std,
                                                         y_test_select_SPECIFIC.astype(np.int64),
                                                         verbose=0)

print("Separate-SPECIFIC validation - Loss: ", score_NN_RandomSearch[0])
print("Separate-SPECIFIC validation - Accuracy: ", score_NN_RandomSearch[1])

Separate-SPECIFIC validation - Loss:  48218.9375
Separate-SPECIFIC validation - Accuracy:  0.692307710647583


#### Sparate Validation - Rest cancer

In [57]:
score_NN_RandomSearch = hypermodel.evaluate(X_test_select_REST_std,
                                                         y_test_select_REST.astype(np.int64),
                                                         verbose=0)

print("Separate-REST validation - Loss: ", score_NN_RandomSearch[0])
print("Separate-REST validation - Accuracy: ", score_NN_RandomSearch[1])

Separate-REST validation - Loss:  70401.03125
Separate-REST validation - Accuracy:  0.5384615659713745


#### Sparate Validation - Primary AML

In [58]:
score_NN_RandomSearch = hypermodel.evaluate(X_val_select_std,
                                                         y_val_select,
                                                         verbose=0)

print("Separate-Primary AML validation - Loss: ", score_NN_RandomSearch[0])
print("Separate-Primary AML validation - Accuracy: ", score_NN_RandomSearch[1])

Separate-Primary AML validation - Loss:  33494.88671875
Separate-Primary AML validation - Accuracy:  0.5833333134651184
